# Example 22: API Documentation Generation

## Learning Objective
Learn to generate comprehensive API documentation for modules and libraries.

## Module-Level Documentation

In [ ]:
"""
User Management API
===================

This module provides user management with CRUD operations.

Quick Start
-----------
    >>> from user_api import UserService
    >>> service = UserService(db)
    >>> user = service.create_user("alice", "alice@example.com")
    >>> print(user.id)
    1

Classes
-------
User
    Data class representing a user.
UserService
    Service for user management operations.
"""

from dataclasses import dataclass
from datetime import datetime
from typing import Optional


@dataclass
class User:
    """A user in the system.
    
    Attributes:
        id: Unique identifier (assigned by database).
        username: Display name (must be unique).
        email: Email address.
        created_at: When the user was created.
        is_active: Whether the account is active.
    
    Example:
        >>> user = User(id=1, username="alice", email="a@b.com", created_at=datetime.now())
        >>> user.is_active
        True
    """
    id: int
    username: str
    email: str
    created_at: datetime
    is_active: bool = True

In [ ]:
class UserService:
    """Service for managing user accounts.
    
    Provides CRUD operations with caching support.
    
    Attributes:
        db: Database connection.
    
    Example:
        >>> service = UserService(database)
        >>> user = service.create_user("bob", "bob@example.com")
        >>> same_user = service.get_user(user.id)  # From cache
    """
    
    def __init__(self, db):
        """Initialize the service.
        
        Args:
            db: Database connection with query/save/delete methods.
        """
        self.db = db
        self._cache = {}
    
    def get_user(self, user_id: int) -> Optional[User]:
        """Retrieve a user by ID.
        
        Checks cache first, then database.
        
        Args:
            user_id: The user's unique identifier.
        
        Returns:
            User object if found, None otherwise.
        
        Example:
            >>> user = service.get_user(123)
            >>> if user:
            ...     print(user.username)
        """
        if user_id in self._cache:
            return self._cache[user_id]
        user = self.db.query(User, user_id)
        if user:
            self._cache[user_id] = user
        return user
    
    def create_user(self, username: str, email: str) -> User:
        """Create a new user.
        
        Args:
            username: Desired username (must be unique).
            email: User's email address.
        
        Returns:
            Newly created User with assigned ID.
        
        Raises:
            ValueError: If username is empty.
            DuplicateError: If username exists.
        """
        user = User(
            id=self.db.next_id(),
            username=username,
            email=email,
            created_at=datetime.now()
        )
        self.db.save(user)
        return user
    
    def delete_user(self, user_id: int) -> bool:
        """Delete a user.
        
        Args:
            user_id: ID of user to delete.
        
        Returns:
            True if deleted, False if not found.
        
        Warning:
            This is irreversible. Consider deactivating instead.
        """
        if user_id in self._cache:
            del self._cache[user_id]
        return self.db.delete(User, user_id)

## Viewing Documentation

In [ ]:
# View docstrings
help(User)
print("\n" + "="*50 + "\n")
print(UserService.create_user.__doc__)

## Documentation Prompts

```
Generate comprehensive API documentation for this module including:
1. Module-level docs with quick start
2. Class docs with usage examples
3. Method docs with parameters and returns
```

```
Generate a README.md for this module with installation,
quick start, and API reference sections.
```

In [ ]:
# Practice: Add documentation to this class
class EmailSender:
    def __init__(self, smtp_host, port, username, password):
        self.host = smtp_host
        self.port = port
        self.username = username
        self.password = password
    
    def send(self, to, subject, body, attachments=None):
        pass
    
    def send_bulk(self, recipients, subject, body):
        pass